In [1]:
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from scipy.stats import zscore

In [2]:
# Load the dataset (modify file path accordingly)
file_path = "Cours_titres.xlsx"
df = pd.read_excel(file_path, index_col=0, parse_dates=True)

# Handle missing values by forward-filling and back-filling
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

returns = df.pct_change().dropna()

### 1. Moving Average

In [3]:
mva_1M = returns.rolling(21).mean().dropna()
mva_3M = returns.rolling(63).mean().dropna()
mva_6M = returns.rolling(126).mean().dropna()
mva = {"mva 1M": mva_1M, "mva 3M" : mva_3M, "mva 6M": mva_6M}

In [4]:
# 12-day and 26-day EMA
ema_12 = returns.ewm(span=12, adjust=False).mean()
ema_26 = returns.ewm(span=26, adjust=False).mean()


### 2. Relative Strength Index

In [5]:
def rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

rsi_14 = returns.apply(rsi, window=14)#.fillna(0)
rsi_14

,ABJC,BICC,BNBC,BOAB,BOABF,BOAC,BOAM,BOAN,BOAS,CABC,...,SNTS,SOGC,SPHC,STAC,STBC,SVOC,TTLC,TTLS,UNLC,UNXC
Date/Société,,,,,,,,,,,,,,,,,,,,,
2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,NaN,100.000000,NaN,NaN,100.000000,100.000000,NaN,NaN,NaN,100.000000,...,0.000000,NaN,0.000000,NaN,100.000000,NaN,0.000000,NaN,NaN,NaN
2013-01-07,NaN,100.000000,NaN,NaN,50.000000,89.889041,NaN,NaN,NaN,100.000000,...,0.000000,NaN,62.101995,NaN,50.943183,NaN,0.000000,NaN,NaN,NaN
2013-01-08,NaN,50.000000,NaN,NaN,50.000000,25.970330,NaN,0.000000,NaN,100.000000,...,0.000000,0.000000,50.859877,NaN,34.681252,NaN,0.000000,NaN,NaN,NaN
2013-01-09,NaN,50.000000,0.000000,NaN,50.000000,68.331101,NaN,50.000000,NaN,100.000000,...,7.932011,50.000000,50.946400,NaN,50.486731,100.0,33.093525,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-12,58.216909,49.322702,40.622123,52.685725,48.635562,30.370707,41.506103,46.420621,45.591764,48.091886,...,39.027158,48.479519,50.006672,58.068148,54.274329,NaN,49.493870,53.302993,50.000000,41.289311
2025-03-13,40.139416,49.953026,56.845298,40.115614,49.775408,27.193603,48.380576,48.243276,54.993811,39.176806,...,32.767018,35.216082,49.405140,50.533792,46.772283,NaN,48.396448,69.624678,66.750851,55.861324
2025-03-14,50.000000,35.544082,49.498932,52.438190,50.134139,33.816298,43.340064,43.675253,50.000000,59.882500,...,48.669079,55.553373,62.002504,46.620982,49.927994,NaN,61.509609,50.786906,50.981618,50.000000
